In [1]:
import pandas as pd

c:\Users\ruoxi\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
data = pd.read_csv('sales.csv')

In [3]:
data

# date -> 0-12 months, 0-3 years, drop unnamed. 

,Unnamed: 0,store_ID,day_of_week,date,nb_customers_on_day,open,promotion,state_holiday,school_holiday,sales
0,425390,366,4,2013-04-18,517,1,0,0,0,4422
1,291687,394,6,2015-04-11,694,1,0,0,0,8297
2,411278,807,4,2013-08-29,970,1,1,0,0,9729
3,664714,802,2,2013-05-28,473,1,1,0,0,6513
4,540835,726,4,2013-10-10,1068,1,1,0,0,10882
...,...,...,...,...,...,...,...,...,...,...
640835,359783,409,6,2013-10-26,483,1,0,0,0,4553
640836,152315,97,1,2014-04-14,987,1,1,0,0,12307
640837,117952,987,1,2014-07-07,925,1,0,0,0,6800
640838,435829,1084,4,2014-06-12,725,1,0,0,0,5344


In [4]:
# Sales -> dependant variable
# 
data.columns

Index(['Unnamed: 0', 'store_ID', 'day_of_week', 'date', 'nb_customers_on_day',
       'open', 'promotion', 'state_holiday', 'school_holiday', 'sales'],
      dtype='object')

In [5]:
data['date'] = pd.to_datetime(data['date'])
data['year'] = pd.to_datetime(data['date']).dt.year
data['month'] = pd.to_datetime(data['date']).dt.month
data

,Unnamed: 0,store_ID,day_of_week,date,nb_customers_on_day,open,promotion,state_holiday,school_holiday,sales,year,month
0,425390,366,4,2013-04-18,517,1,0,0,0,4422,2013,4
1,291687,394,6,2015-04-11,694,1,0,0,0,8297,2015,4
2,411278,807,4,2013-08-29,970,1,1,0,0,9729,2013,8
3,664714,802,2,2013-05-28,473,1,1,0,0,6513,2013,5
4,540835,726,4,2013-10-10,1068,1,1,0,0,10882,2013,10
...,...,...,...,...,...,...,...,...,...,...,...,...
640835,359783,409,6,2013-10-26,483,1,0,0,0,4553,2013,10
640836,152315,97,1,2014-04-14,987,1,1,0,0,12307,2014,4
640837,117952,987,1,2014-07-07,925,1,0,0,0,6800,2014,7
640838,435829,1084,4,2014-06-12,725,1,0,0,0,5344,2014,6


In [6]:
data.drop(columns = ['Unnamed: 0'], inplace =True)

In [7]:
'''Before replacing "a, b, c" into 1, we keep it for the model testing.'''
#data['year'].unique()
#data['state_holiday'] = data['state_holiday'].apply(lambda x: 1 if x in ['a', 'b', 'c'] else 0)
#data['state_holiday'].unique()

'Before replacing "a, b, c" into 1, we keep it for the model testing.'

In [8]:
data['year'].value_counts()

year
2013    256337
2014    235602
2015    148901
Name: count, dtype: int64

In [9]:
data['store_ID'].value_counts()

store_ID
1045    645
309     636
754     635
432     634
286     634
       ... 
1004    448
287     448
1065    445
81      438
542     436
Name: count, Length: 1115, dtype: int64

In [10]:
data.isna().sum()

store_ID               0
day_of_week            0
date                   0
nb_customers_on_day    0
open                   0
promotion              0
state_holiday          0
school_holiday         0
sales                  0
year                   0
month                  0
dtype: int64

Instead, we first do one-hot encoding on "day_of_week", "state_holiday", "year", "month"

In [11]:

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False, drop='first') 

# Fit and transform the categorical columns
encoded_data = encoder.fit_transform(data[["day_of_week", "state_holiday", "year", "month"]])

encoded_data

c:\Users\ruoxi\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
# Create a DataFrame with the encoded data and column names
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(["day_of_week", "state_holiday", "year", "month"]))

# Concatenate the encoded DataFrame with the original DataFrame
data_encoded = pd.concat([data, encoded_df], axis=1)

# Drop the original categorical columns if needed
data_encoded = data_encoded.drop(columns =["day_of_week", "state_holiday", "year", "month"], axis=1)

In [13]:
data_encoded.columns

Index(['store_ID', 'date', 'nb_customers_on_day', 'open', 'promotion',
       'school_holiday', 'sales', 'day_of_week_2', 'day_of_week_3',
       'day_of_week_4', 'day_of_week_5', 'day_of_week_6', 'day_of_week_7',
       'state_holiday_a', 'state_holiday_b', 'state_holiday_c', 'year_2014',
       'year_2015', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12'],
      dtype='object')

In [14]:
#drop "date" as well
data_encoded = data_encoded.drop(columns =['date'], axis=1)

We first put all the potential independent varibles in the basic Linear Regression Model


In [15]:
# create our predictor and response
predictor = data_encoded[['nb_customers_on_day', 'open', 'promotion',
       'school_holiday', 'day_of_week_2', 'day_of_week_3',
       'day_of_week_4', 'day_of_week_5', 'day_of_week_6', 'day_of_week_7',
       'state_holiday_a', 'state_holiday_b', 'state_holiday_c', 'year_2014',
       'year_2015', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12']]
response = data_encoded['sales']

In [16]:
# Train Test data split with 20% as the test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictor, response, test_size=0.2, random_state = 1)

In [17]:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Initializing and fitting the model
model = LinearRegression()
result = model.fit(X_train, y_train)

# Calculating the regression line
regression_line = result.predict(X_train)

y_pred = model.predict(X_test)
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.8608515288695021
0.8626206509518384


Basic Linear Regression Model gives 86% accuracy score, and there's no over/under fitting

Let's use Lasso to see if there are some variables that are not really predictive 

In [18]:
from sklearn.linear_model import Lasso
regr = Lasso(alpha=1.0)

# Train the linear regression
model = regr.fit(X_train, y_train)

# Score the model
print(model.score(X_test, y_test))

model.coef_

0.8624439615340588


array([    6.01122171,  1007.81681086,  1420.90093474,   168.53874893,
        -535.49664098,  -701.33642134,  -750.22457885,  -595.4752879 ,
        -266.86144682, -1213.69841339, -1116.72475856, -1616.66613663,
       -1172.96141501,    66.62942487,   262.84164561,   -31.98945287,
          41.93055515,    59.57414681,    99.72009472,   133.90681895,
          25.63496802,   -97.75998623,   -71.33234044,   -71.75781224,
         191.4078938 ,   894.31041787])

Lasso gives the same accurary score 86% as the basic linear regression model. For Linear Regression, we indeed need to include all the variables we have in the basic linear regression model

Thirdly, we try KNN regressor

In [19]:
from sklearn.neighbors import KNeighborsRegressor


In [20]:
# create knn model
knnr = KNeighborsRegressor(n_neighbors = 3,weights='distance')

# Train the knn model
model = knnr.fit(X_train, y_train)

# Because the computing time is really long, I only did the accuracy score for the test dataset
print(knnr.score(X_test, y_test))

0.8140676178074384


KNN Regressor with 3 neighbors gives 81% accuracy scores

In [21]:
knnr_5 = KNeighborsRegressor(n_neighbors = 5,weights='distance')

# Train the knn model
knnr_5.fit(X_train, y_train)

# Because the computing time is really long, I only did the accuracy score for the test dataset
print(knnr.score(X_test, y_test))

0.8140676178074384


KNN Regressor with 5 neighbors gives 82% accuracy scores

Fourthly, we use Basic Decision Tree Regressor and we include "store_ID"

In [22]:
predictor_all = data_encoded[['store_ID', 'nb_customers_on_day', 'open', 'promotion',
       'school_holiday', 'day_of_week_2', 'day_of_week_3',
       'day_of_week_4', 'day_of_week_5', 'day_of_week_6', 'day_of_week_7',
       'state_holiday_a', 'state_holiday_b', 'state_holiday_c', 'year_2014',
       'year_2015', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12']]

# And we split the data into train and test again
X_train, X_test, y_train, y_test = train_test_split(predictor_all, response, test_size=0.2, random_state = 1)

In [23]:
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor(random_state=1) 
tree.fit(X_train, y_train)
print(tree.score(X_train,y_train))
print(tree.score(X_test,y_test))

0.9999884175563035
0.907435928283098


Basic Decision Tree Regressor gives about 91% accuracy, but the training dataset is overfitted. Let's try the cross validation method

In [24]:
from sklearn.model_selection import cross_validate

model = DecisionTreeRegressor(random_state=1)
model.fit(X_train, y_train)
results = cross_validate(model,X_train, y_train, cv = 10)
print(results['test_score'].mean())
print(model.score(X_test,y_test))

0.9007862907588301
0.907435928283098


With cross validation, we got the Decision Tree Regressor having about 91% accuracy score

Fifthly, we use Bagging

In [25]:
from sklearn.ensemble import BaggingRegressor

bagging_reg = BaggingRegressor(
    DecisionTreeRegressor(max_depth=5), 
    n_estimators=30, 
    max_samples=100, 
    random_state=1)

bagging_reg.fit(X_train, y_train)
print(bagging_reg.score(X_train,y_train))
print(bagging_reg.score(X_test,y_test))

0.8520892905472692
0.8534197714568175


Bagging for Decision Tree Regressor only gives 86% accuracy

Sixthly, we try Random Forest

In [26]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators=30,
                               max_depth=5, 
                               random_state=1) 

forest.fit(X_train, y_train)

print(forest.score(X_train,y_train))
print(forest.score(X_test,y_test))

0.8613018286420235
0.8619376443385338


Random Forest gives 86% accuracy score

Seventhly, we try AdaBoost

In [27]:
from sklearn.ensemble import AdaBoostRegressor

ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=5), 
                            n_estimators=30, 
                            random_state=1 
                            )
ada_reg.fit(X_train, y_train)
print(ada_reg.score(X_train,y_train))
print(ada_reg.score(X_test,y_test))

0.837811158623817
0.838952866411886


Eightly, we try Gradient boosting

In [28]:
from sklearn.ensemble import GradientBoostingRegressor

gb_reg = GradientBoostingRegressor(max_depth=5, 
                                   n_estimators=30,
                                   random_state=1 
                                   )
gb_reg.fit(X_train, y_train)
print(gb_reg.score(X_train,y_train))
print(gb_reg.score(X_test,y_test))

Gradient booting gives 88% accuracy score

Last, we try Extreme Gradient Boosting

In [ ]:
!pip install xgboost

import xgboost

xgb_reg = xgboost.XGBRegressor(max_depth=5)
xgb_reg.fit(X_train, y_train)
print(xgb_reg.score(X_train,y_train))
print(xgb_reg.score(X_test,y_test))


0.9245322742837572
0.9237774783104511


This is the highest score we have so far, so we will save this as our model pickle

We want to know if we have less predictors to make a simpler model to explain to the stakeholders, how the prediction would be

In [ ]:
simple_predictor = data_encoded[['nb_customers_on_day','open','promotion']]
X_train, X_test, y_train, y_test = train_test_split(simple_predictor, response, test_size=0.2, random_state = 1)

xgb_reg_simple = xgboost.XGBRegressor(max_depth=5)
xgb_reg_simple.fit(X_train, y_train)
print(xgb_reg_simple.score(X_train,y_train))
print(xgb_reg_simple.score(X_test,y_test))


0.8644737798694812
0.8661446034343925


XGB Regressor gives more than 92% accuracy without overfitting the training dataset

In [ ]:
import pickle

pickle.dump(xgb_reg, open('model.pkl', 'wb'))


In [ ]:
pickle.dump(xgb_reg_simple, open('simple_model.pkl', 'wb'))

In [ ]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
